In [1]:
import pandas as pd
from cassandra.cluster import Cluster

cluster = Cluster(['192.168.7.110', '192.168.7.111', '192.168.7.61'])
#session = cluster.connect('data.storageinfo')
session = cluster.connect()

rows = session.execute('select loc from data.storageinfo limit 5')
for row in rows:
    print(row.loc.split('\u0001'))
    #print(type(row.loc))
    #print(row.loc)

cluster.shutdown()

['192.168.8.138', '3306', 'mysql_native_password', 'userinfo9', 'username']
['192.168.8.138', '3306', 'mysql_native_password', 'userinfo7', 'username']
['192.168.8.138', '3306', 'mysql_native_password', 'userinfo8', 'address']
['192.168.8.138', '3306', 'mysql_native_password', 'userinfo8', 'address']
['192.168.8.71', '3306', 'mysql_native_password', 'a', 'address']


In [3]:
from cassandra.cluster import Cluster

def cs_select(cs_host_list, csql_str):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(csql_str)
    cluster.shutdown()
    return res
    
#rows = cs_select(['192.168.7.110', '192.168.7.111'], "select * from api.ds_services where ip='192.168.8.74' and port=1433")
#[print(row.ip, type(row.files), type(row.badges), row.connection, row.port) for row in rows]

DS_DELIMITER = "\u0001"
sel_ip = "192.168.7.71"
sel_port = 1433
sel_l = DS_DELIMITER.join([sel_ip, str(sel_port)]) + DS_DELIMITER
print(sel_l)
sel_u = DS_DELIMITER.join([sel_ip, str(sel_port+1)]) + DS_DELIMITER
print(sel_u)

#query = "select * from scanner.sqlcatalog"# where loc LIKE '%192.168.7.71%'"
#rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
#[':'.join([row.db_name, row.table_name, row.col_name]) for row in rows if sel in row.loc]

query = "select * from scanner.sqlcatalog where loc >= '" + sel_l + "' and loc < '" + sel_u + "' allow filtering"
rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
[':'.join([row.ip, str(row.port), row.db_name, row.table_name, row.col_name]) for row in rows]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select sum(dbs) as dbs, sum(tables) as tables, sum(files) as files from api.ds_services')
#[row.dbs for row in rows][0]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select count(ip) as servers, sum(services) as services from api.ds_servers')
#[row.servers for row in rows][0]
#servers, services = [(row.servers, row.services) for row in rows][0]
#print(servers, services)
#[print(type(row.ip)) for row in rows]

192.168.7.711433
192.168.7.711434


['192.168.7.71:1433:test_5:testtable:content',
 '192.168.7.71:1433:tempdb:#BA2D01C7:job_count',
 '192.168.7.71:1433:huaPIITest:pii_3rd:tab3col1',
 '192.168.7.71:1433:tempdb:#BB8DF592:subsystem',
 '192.168.7.71:1433:tempdb:#A2C247C8:maj',
 '192.168.7.71:1433:test_101:testtable:Id',
 '192.168.7.71:1433:tempdb:#BFE5DB1D:objname',
 '192.168.7.71:1433:huaPIITest:pii_4th:tab4col1',
 '192.168.7.71:1433:test_9:testtable:phonenumber',
 '192.168.7.71:1433:huaPIITest:pii_4th:city',
 '192.168.7.71:1433:test_11:testtable:Id',
 '192.168.7.71:1433:test_2:testtable:content',
 '192.168.7.71:1433:tempdb:#BFE5DB1D:name',
 '192.168.7.71:1433:tempdb:#BFE5DB1D:value',
 '192.168.7.71:1433:test_100:testtable:Id',
 '192.168.7.71:1433:test_10:testtable:phonenumber',
 '192.168.7.71:1433:master:Employees:name',
 '192.168.7.71:1433:test_100:Persons:PersonID',
 '192.168.7.71:1433:tempdb:#B9A5AD20:subsystem_id',
 '192.168.7.71:1433:test_8:testtable:name',
 '192.168.7.71:1433:test_1:testtable:source',
 '192.168.7.71:

In [4]:
query = "select ddn_name, city, count(pii_record) as value from api.content_locations group by ddn_name, city"
query = "select ddn_name, city, count(last_seen_time) as value from api.access_locations group by ddn_name, city"
#query = "select city, pii_record from api.content_locations where ddn_name = 'aaa' allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"ddn_name": row.ddn_name,
            "content": row.city,
            "value": row.value} for row in rows]
#print(cs_data)
#if cs_data:
#    print("aaa")
#else:
#    print("bbb")
cs_data = [
    {"ddn_name": "hr", "content": "Beijing", "value": 5},
    {"ddn_name": "hr", "content": "Tianjin", "value": 1},
    {"ddn_name": "sales", "content": "Shanghai", "value": 5},
    {"ddn_name": "sales", "content": "Beijing", "value": 22},
    {"ddn_name": "hr", "content": "Nanjing", "value": 1},
]

df = pd.DataFrame(cs_data)
d = df.groupby(['content'])['value'].sum().to_dict()
cs_data = [{"content": k, "value": v} for k, v in d.items()]
cs_data

[{'content': 'Beijing', 'value': 27},
 {'content': 'Nanjing', 'value': 1},
 {'content': 'Shanghai', 'value': 5},
 {'content': 'Tianjin', 'value': 1}]

In [5]:
query = "select ddn_name, feature_name as content, feature_value as value from api.pii_special"
cs_data = [{"content": row.content,
            "value": row.value} for row in rows]
cs_data

[]

In [6]:
query = "select ddn_name as content, bytes as x, frequency as y from api.ddn_vol_freq"
cs_data = [{"content": row.content,
            "x": row.x,
            "y": row.y} for row in rows]
cs_data = [
    {"content": "hr", "x": 5, "y": 5},
    {"content": "hr", "x": 1, "y": 5},
    {"content": "sales", "x": 5, "y": 5},
    {"content": "sales", "x": 2, "y": 5},
    {"content": "hr", "x": 1, "y": 5},
]
df = pd.DataFrame(cs_data)
dx = df.groupby(['content'])['x'].sum().to_dict()
dx
dy = df.groupby(['content'])['y'].sum().to_dict()
dy
d = df.groupby(['content'])['x', 'y'].sum().to_dict()
d
#{'x': {'hr': 7, 'sales': 7}, 'y': {'hr': 15, 'sales': 10}}
keys = list(d['x'].keys())
keys
#cs_data = [{"content": k, "value": v} for k, v in d.items()]

cs_data = [{"content": key, "x": d['x'][key], "y": d['y'][key]} for key in keys]
cs_data

[{'content': 'hr', 'x': 7, 'y': 15}, {'content': 'sales', 'x': 7, 'y': 10}]

In [9]:
#query = "select ip, port from api.ds_services where ip='192.168.7.71' and port=80"
query = "select ip, port from api.ds_services where port=80 allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
[print(row.ip, row.port) for row in rows]

192.168.8.71 80
192.168.7.71 80


[None, None]

In [10]:
def cs_update(cs_host_list, tab_n, upg_d, sel_d):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    upg_str = ",".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in upg_d.items()])
    sel_str = " and ".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in sel_d.items()])
    csql_str = "update " + tab_n + " set " + upg_str + " where " + sel_str
    print(csql_str)
    res = session.execute(csql_str)
    session.shutdown()
    cluster.shutdown()
    return res

cs_update(['192.168.7.110', '192.168.7.111', '192.168.7.61'], 
          "api.ds_services", 
          {"connection": False, "status": "Pending"}, 
          {"ip": "192.168.7.71", "port": 80})

update api.ds_services set connection=False,status='Pending' where ip='192.168.7.71' and port=80
